<a href="https://colab.research.google.com/github/OliveiraMatheuss/Automacao_Rampage/blob/main/automacao_comprovantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Leitor de comprovantes bancários 

## Iniciar Script

In [ ]:

# Instalações
!sudo apt-get install tesseract-ocr-por | grep -v 'already satisfied'
!pip install pytesseract  | grep -v 'already satisfied'
!pip install gspread_dataframe==3.3.0  | grep -v 'already satisfied'
!pip install gspread==5.4.0  | grep -v 'already satisfied'

#------Importações de Pacotes -----------------------------------------------------------------------------------|

import pytesseract
import pandas as pd
import re
from tqdm import tqdm
import glob
from PIL import Image
from oauth2client.client import GoogleCredentials 
from google.auth import default
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe

#------------------------------------------------------------------------------------------------------------------------#

#------------------------------------------------------------------------------------------------------------------------|

# Montar o drive para buscar os arquivos
from google.colab import drive
drive.mount('/content/drive') #Acessar drive da Rampage

#------------------------------------------------------------------------------------------------------------------|
# Inicio Função de leitura dos comprovantes -----------------------------------------------------------------------------|

def tabela_pix():
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    print('Cole o Caminho onde estão os comprovantes\n')
    path = input()
    diretorio = glob.glob(path + '/*.jpg')

    print('\n Digite o nome da tabela \n')
    name_tab = input()

    arquivo = []
    entrada = []
    saida = []
    data_transacao = []
    descricao = []
    pagador = []

    for nome in tqdm(diretorio):
        
        #Leitura da imagem
        diretorio = nome
        image_path_in_colab = nome

        #Extrai a informaçõe em formato de texto
        extractedInformation = pytesseract.image_to_string(Image.open(image_path_in_colab),lang="por")
        texto = extractedInformation
        manipula_arquivo = nome.split('/')
        manipula_arquivo = manipula_arquivo[-1] # Pega somente o nome do arquivo e ignora o diretório
        arquivo.append(manipula_arquivo) # guarda o nome do arquivo em uma lista
            
        # Buscar  valor Recebido
        fatia = texto.split() 

       

        try:
          #Tratativa de erro, na leitura do comproavante pode haver comprovantes com R$ ou com RS 
          
          indice_RS = fatia.index('RS')
        except ValueError:
          indice_RS = fatia.index('R$')


        # Encontrar os tipos de entrada( receber uma transferência)
        if (('Pixrecebido' in texto) 
              or ('Pix recebido' in texto) 
              or ('Transferência recebida' in texto) 
              or ('Cashback Promocional' in texto) 
              or ('Cashback' in texto) 
              or ('Boleto recebido 48 horas' in texto) 
              or ('Transferência recebida' in texto)
                  ):  
            
            entrada.append(fatia[indice_RS+1]) # Valor do Pix Recebido
            saida.append(' ') # vazio pois é uma entrada
            fatia = texto.split('\n')
            fatia = [elemento for elemento in fatia if elemento.strip() != ""]
            if 'Descrição' in texto:
              
                
              if 'Descrição' in fatia[fatia.index('Quem enviou')-1]:
                    
                  descricao.append(fatia[fatia.index('Quem enviou')-1][9:])
                
              else: 
                 descricao.append(fatia[fatia.index('Quem enviou')-1])
            else: 
              descricao.append('sem descricao')

            
        # Encontrar os tifpo de saida 
        elif (('—Compra-crédito debitado' in texto) 
                or ('Pix enviado' in texto) 
                or ('Pix enviado devolvido' in texto) 
                or ('Pagamento efetuado' in texto) 
                or ('Compra no débito' in texto) 
                or ('Compra - crédito debitado' in texto) 
                or ('Transferência enviada' in texto)
                    ):
          
            entrada.append(' ')
            saida.append(fatia[indice_RS+1])

            fatia = texto.split('\n')
            fatia = [elemento for elemento in fatia if elemento.strip() != ""]

            if 'Descrição' in texto:


                  if 'Descrição' in fatia[fatia.index('Quem recebeu')-1]:
                    
                      descricao.append(fatia[fatia.index('Quem recebeu')-1][9:])
                
                  else: 
                    descricao.append(fatia[fatia.index('Quem recebeu')-1])

            else: 
                descricao.append('sem descricao')

        # Se o tipo de transferência não foi especificado aqui
        else:
              entrada.append('tipo de pagamento não suportado')
              saida.append('tipo de pagamento não suportado')
            
        #Buscar a data do Pagamento
        matchs = re.search(r'(\d+/\d+/\d+)', texto)
        data_transacao.append(matchs.group(1)) # Data
            
        # Buscar nome do Pagador
        fatia = texto.split('\n')
        fatia = [elemento for elemento in fatia if elemento.strip() != ""]

        if ('Cashback Promocional' in texto) or ('Cashback' in texto):
          pagador.append('Cashback')
        elif 'Boleto recebido 48 horas' in texto:
          pagador.append('Boleto recebido')
        else:
          pagador.append(fatia[fatia.index('Sobre a transação')-1]) #Nome Pagador
            
      


    a = {'Nome':pagador, 'data': data_transacao, 'entrada': entrada, 'Saida':saida, 'Descrição': descricao, 'arquivo':arquivo}
    df = pd.DataFrame.from_dict(a, orient='index') 
    df = df.transpose()

    #Cria a planilha que vai receber os dados
    """
     para pegar o folder id : abra a pasta no drive e copia a ultima parte do link 
     https://drive.google.com/drive/u/0/folders/<folder_id> 

    """

    folder_id = '1ND0LulP2nnHGGC-QRRV34uSt5Hwt1qnR'
    gc.create(name_tab, folder_id= folder_id)
    
    #Abre a planilha via Python
    gs = gc.open(name_tab)
    worksheet1 = gs.worksheet('Página1') # Nome da pagina padrão ao criar uma planilha
    worksheet1.clear()
    set_with_dataframe(worksheet=worksheet1, dataframe=df, include_index=False)

    return df

# Fim da Função -------------------------------------------------------------------------------------------------------

# Inicio das entradas ------------------------------------------------------------------------------------------------


Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr-por is already the newest version (1:4.00~git30-7274cfa-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Gerar Tabela

In [ ]:
df = tabela_pix()

## Contato

* Autor  : Matheus Ramos de Oliveira

* contato: ramos.matheus@engenharia.ufjf.br

* LinkedIn: https://www.linkedin.com/in/oliveiramatheuss/




